In [2]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
from pmdarima.arima import ndiffs
import pmdarima as pm
from sklearn.metrics import mean_squared_error
from pandas import datetime
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action='ignore')
air_hour = pd.read_csv('./air_hour.csv')
air_hour.set_index('date', inplace = True)
air_hour.index = pd.to_datetime(air_hour.index)
air_hour_1 = air_hour.iloc[7::8]
air_hour_1.index = pd.to_datetime(air_hour_1.index)
air_hour_2=air_hour_1.astype(np.float64)

In [5]:
columns = ["AK_close", "korea_air_close", "aisana_close", "airbusan_close", "jeju_close", "jin_close", "tway_close" ]
title = ["AK 홀딩스(006840)", "대한항공(003490)", "아시아나항공(020560)", "에어부산(298690)", "제주항공(089090)", "진에어(272450)", "티웨이항공(091810)"]
orders = [[3,1,1],[0,1,2], [1,1,0], [3,1,3], [2,1,3], [0,1,0], [0,1,0]]

for i in range(0,7,1):

    a = title[i]
    y_train = air_hour_2[columns[i]].iloc[:188]
    y_test = air_hour_2[columns[i]].iloc[188:]
    
    from pmdarima.arima import ndiffs
    import pmdarima as pm

    #차분 차수 찾기
    kpss_diffs = ndiffs(y_train, alpha=0.05 , test='kpss', max_d = 6)
    adf_diffs = ndiffs(y_train, alpha=0.05, test='adf', max_d=6)
    n_diffs = max(adf_diffs,kpss_diffs)

    n_diffs_print = f'적정 차분 차수는 {n_diffs}'
    
    print(n_diffs_print)
 
    from statsmodels.tsa.arima_model import ARIMA
    
    model = pm.arima.ARIMA(order = orders[i])
    model.fit(y_train)
    
    
    def forcast_one_step():
        fc, conf_int = model.predict(n_periods=1, return_conf_int=True)
    
        return (fc.tolist()[0] , np.asarray(conf_int).tolist()[0] )
    forcast_one_step()
    
    forcast_list=[]
    y_pred = []
    pred_upper=[] 
    pred_lower = []
    
    for i in y_test:
      fc , conf = forcast_one_step()
    
      y_pred.append(fc)
      pred_upper.append(conf[1])
      pred_lower.append(conf[0])
      
    
      model.update(i)
    
    print(f"평균 절대 비율 오차: {np.mean(np.abs((y_test - y_pred) / y_test)) * 100:.2f}%")
    print(f"평균 괴리율: {np.mean((y_pred - y_test) / y_test) *100:.3f} %")
    
    import plotly.graph_objects as go
    fig = go.Figure([
        go.Scatter(x=y_train.index , y=y_train, name='Train', mode='lines',line=dict(color='royalblue')),
        #테스트데이터
        go.Scatter(x=y_test.index , y=y_test, name='Test', mode='lines',line=dict(color='red')),
        #predict 데이터
        go.Scatter(x=y_test.index , y=y_pred, name='Prediction', mode='lines',line=dict(color='yellow', dash='dot', width=3)),
        #신뢰구간
        go.Scatter(x=y_test.index.tolist() + y_test.index[::-1].tolist(), 
                y= pred_upper + pred_lower[::-1],
               fill='toself',
                fillcolor='rgba(0,0,30,0.1)',
                line={'color':'rgba(0,0,0,0)'}, 
                hoverinfo="skip",
                showlegend=True
              )
                ])

    fig.update_layout(height=400 , width=1000, title_text=a)
    fig.show()
    print("="*130)
  

적정 차분 차수는 1
평균 절대 비율 오차: 2.05%
평균 괴리율: -0.274 %


적정 차분 차수는 1
평균 절대 비율 오차: 1.32%
평균 괴리율: 0.015 %


적정 차분 차수는 1
평균 절대 비율 오차: 2.09%
평균 괴리율: 0.091 %


적정 차분 차수는 1
평균 절대 비율 오차: 1.80%
평균 괴리율: -0.018 %


적정 차분 차수는 1
평균 절대 비율 오차: 2.50%
평균 괴리율: -0.259 %


적정 차분 차수는 1
평균 절대 비율 오차: 2.35%
평균 괴리율: -0.094 %


적정 차분 차수는 1
평균 절대 비율 오차: 2.10%
평균 괴리율: -0.020 %
